
<img align="left" src = https://www.linea.org.br/wp-content/themes/LIneA/imagens/logo-header.png width=100 style="padding: 30px"> 
<img align="left" src = https://cdn2.webdamdb.com/1280_c3PXjCZbPM23.png width=180> <!-- style="padding: 20px"--> 

# Training Set Maker - Tutorial Notebook

**Contact author**: Julia Gschwend ([julia@linea.org.br](mailto:julia@linea.org.br)) 

**Last verified run**: yyyy-mm-dd <br>



# Notebook under construction - please wait for the first verified version

Photo-z (PZ) Training Set Maker (TSM) - [pz-tsm](https://github.com/linea-it/pz-tsm) - is tool available in the PZ Server library to support the creation of Training Sets for catalog-level machine-learning-based photo-z algorithms using spec-z (or true z, for simulations) catalogs provided by the [PZ Server](https://github.com/linea-it/pz-server) and photometric data from LSST Objects Catalogs. 

# classes para prod types - specz cat, training set (herda do pd.dataframe) 
# pz server traz objetos já na classe correspondente
### atributos: prod id, metadata
# tsm tem funções 
### combine specz catalogs - recebe lista de objetos specz catalogs ou lista de prod ids e criterios como argumento
### matching - recebe 1 specz e 1 photo set (datafranme qualquer local) ou 1 prod id e 1 release tag (job remoto)
***
